In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical

import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
learning_rate = 0.001
training_epochs = 6
batch_size = 50

In [3]:
cur_dir = os.getcwd()
ckpt_dir_name = 'checkpoints'
model_dir_name = 'cnn_session_mode'

checkpoint_dir = os.path.join(cur_dir, ckpt_dir_name, model_dir_name)
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, model_dir_name+'.ckpt')
## TensorBoard

In [5]:
mnist = keras.datasets.mnist

(train_x, train_y), (test_x, test_y) = mnist.load_data()

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [6]:
train_x = train_x.astype(np.float32) / 255
test_x = test_x.astype(np.float32) / 255

train_x = np.expand_dims(train_x,3)
test_x = np.expand_dims(test_x,3)

train_y = to_categorical(train_y, 10)
test_y = to_categorical(test_y, 10)


train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(buffer_size=70000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(batch_size)

num_train_data = train_x.shape[0]
num_test_data = test_x.shape[0]

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(60000, 28, 28, 1) (60000, 10) (10000, 28, 28, 1) (10000, 10)


In [7]:
# Traditional Way
class Model:
    def __init__(self):
        self.X = tf.placeholder(tf.float32, [None, 28, 28, 1])
        self.Y = tf.placeholder(tf.float32, [None,10])
        self.is_training = tf.placeholder(tf.bool, [])
    
    def create_model(self, reuse=False):
        self.model = self._model(self.X, self.is_training, reuse=reuse)
        
    def _model(self, x, is_training=False, reuse=False):
        with tf.variable_scope('model_layers', reuse=reuse):
            conv1 = tf.contrib.layers.conv2d(x, 32, 3, padding='SAME', scope="conv1", activation_fn=tf.nn.relu)
            # (n, 28, 28, 1) => (n, 28, 28,32)
            pool1 = tf.contrib.layers.max_pool2d(conv1, 2)
            # (n, 28, 28, 32) => (n, 14, 14,32)
            
            conv2 = tf.contrib.layers.conv2d(pool1, 64, 3, padding='SAME',scope='conv2', activation_fn=None)
            # (n, 14, 14, 32) => (n, 14, 14, 64)
            batch2 = tf.contrib.layers.batch_norm(conv2, is_training=is_training, scope="batch2", activation_fn=tf.nn.relu)
            pool2 = tf.contrib.layers.max_pool2d(batch2, 2)
            # (n, 14, 14, 64) => (n, 7, 7, 64)
            
            conv3 = tf.contrib.layers.conv2d(pool2, 128, 3, padding='SAME', scope='conv3', activation_fn=tf.nn.relu)
            # (n, 7, 7, 64) => (n, 7, 7, 128)
            flatten = tf.contrib.layers.flatten(conv3, scope='flatten')
            flatten = tf.contrib.layers.dropout(flatten, keep_prob=0.7, is_training=is_training)
            
            # fully-connected-layer
            fc_layer1 = tf.contrib.layers.fully_connected(flatten, 256, activation_fn=None, scope='fc_layer1')
            batch4 = tf.contrib.layers.batch_norm(fc_layer1, is_training=is_training, scope='batch4', activation_fn=tf.nn.relu)
            batch4 = tf.contrib.layers.dropout(batch4, keep_prob=0.7, is_training=is_training)
            
            output = tf.contrib.layers.fully_connected(batch4, 10, activation_fn=None, scope='output_layer')
            
            return output

In [8]:
cnn = Model()
cnn.create_model()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=cnn.model, labels= cnn.Y))

pred = tf.equal(tf.argmax(cnn.model,1), tf.argmax(cnn.Y, 1))
acc = tf.reduce_mean(tf.cast(pred, tf.float32))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, epsilon=0.01)

var_list=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='model_layers')

grads_and_vars = optimizer.compute_gradients(loss, var_list = var_list)
train = optimizer.apply_gradients(grads_and_vars)


saver = tf.train.Saver(var_list)

In [10]:
init = tf.global_variables_initializer()
num_train_data = train_x.shape[0]
num_test_data = test_x.shape[0]

train_iterator = train_dataset.make_initializable_iterator()
test_iterator = test_dataset.make_initializable_iterator()

tr_x, tr_y = train_iterator.get_next()
ts_x, ts_y = test_iterator.get_next()

In [11]:
with tf.Session() as sess:
    
    try:
        saver.restore(sess, checkpoint_path)
        print('Variables are succesfully resotored')
    except:
        sess.run(init)
        print('Failed to resotre variables from checkpoints')
        
    for epoch in range(training_epochs):
        avg_loss = 0
        avg_train_acc = 0
        avg_test_acc = 0
        train_step = 0
        test_step = 0
        
        
        # training set
        sess.run(train_iterator.initializer)
        for _ in range(num_train_data//batch_size):
            train_inputs, train_labels = sess.run([tr_x, tr_y])
            feed_dict={cnn.X:train_inputs, cnn.Y: train_labels, cnn.is_training:True}
            step_loss, step_acc, _ = sess.run([loss, acc, train], feed_dict=feed_dict)
            avg_loss += step_loss
            avg_train_acc += step_acc
            train_step += 1
        avg_loss /= train_step
        avg_train_acc /= train_step
            
        print("{}th Epoch".format(epoch+1))
        print("train_avg_loss: {}\t train_avg_acc: {}".format(avg_loss, avg_train_acc))
        
        if (epoch+1)%5 == 0:
            # test_set
            sess.run(test_iterator.initializer)
            for _ in range(num_test_data//batch_size):
                test_inputs, test_labels = sess.run([ts_x, ts_y])
                step_acc = sess.run(acc, feed_dict={
                    cnn.X:test_inputs, 
                    cnn.Y:test_labels,
                    cnn.is_training:False})
                avg_test_acc += step_acc
                test_step += 1
            avg_test_acc /= test_step
            print('test_avg_acc: {}'.format(avg_test_acc))
            saver.save(sess, checkpoint_path)


Instructions for updating:
Use standard file APIs to check for files with this prefix.
Failed to resotre variables from checkpoints
1th Epoch
train_avg_loss: 0.17217717277933844	 train_avg_acc: 0.9493166678460936
2th Epoch
train_avg_loss: 0.056719522549925995	 train_avg_acc: 0.9833500038584073
3th Epoch
train_avg_loss: 0.04086129269960414	 train_avg_acc: 0.9872000045577685
4th Epoch
train_avg_loss: 0.0331407292530154	 train_avg_acc: 0.9898666708171368
5th Epoch
train_avg_loss: 0.026944331123522715	 train_avg_acc: 0.9914500041306019
test_avg_acc: 0.8620999950170517
6th Epoch
train_avg_loss: 0.02363658334109156	 train_avg_acc: 0.9923500040173531
